# Save Halo Mergers
Nithun Selva, August 2025

This notebook assumes that you have the output from `run_Anna_startrace.ipynb` available.

### Import

In [2]:
%matplotlib inline
import os
import socket
hostname = socket.gethostname()
if 'emu' in hostname:
    os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/ns1917/tangos_sims/'
    # os.environ['TANGOS_DB_CONNECTION'] = '/home/ns1917/Databases/Marvel_BN_N10.db'
    os.environ['TANGOS_DB_CONNECTION'] = '/home/ns1917/pynbody/Tangos/Marvel_BN_N10.db'
    os.chdir('/home/ns1917/pynbody/AnnaWright_startrace/')
else: # grinnell
    os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
    # os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'
    os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/pynbody/Tangos/Marvel_BN_N10.db'
    os.chdir('/home/selvani/MAP/pynbody/AnnaWright_startrace/')

import pynbody
import numpy as np
import h5py
import math
import tangos as db
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import glob
from pynbody.array import SimArray
import pandas as pd
import tqdm.auto as tqdm

In [3]:
# Simulation name and path
if 'emu' in hostname:
    simpath = '/home/ns1917/tangos_sims/'
    outfile_dir = "/home/ns1917/pynbody/stellarhalo_trace_aw/"
else:
    simpath = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
    outfile_dir = "/home/selvani/MAP/pynbody/stellarhalo_trace_aw/"

# basename = 'cptmarvel.cosmo25cmb.4096g5HbwK1BH'
# basename = 'rogue.cosmo25cmb.4096g5HbwK1BH'
basename = 'storm.cosmo25cmb.4096g5HbwK1BH'
# ss_dir = 'cptmarvel.4096g5HbwK1BH_bn'#'snapshots_200crit_h329' # same as db_sim?
# ss_dir = 'rogue.4096g5HbwK1BH_bn'
ss_dir = 'storm.4096g5HbwK1BH_bn'
sim_base = simpath + ss_dir + '/'
ss_z0 = sim_base + basename + '.004096'

In [4]:
# Read in data from Anna's pipeline
with h5py.File(outfile_dir+'/'+basename+'_allhalostardata_upd.h5','r') as f:
    hostids = f['host_IDs'].asstr()[:] # unique host IDs
    partids = f['particle_IDs'][:] # iords
    pct = f['particle_creation_times'][:] # formation times
    ph = f['particle_hosts'][:] # local host IDs (i.e., host at formation time)
    pp = f['particle_positions'][:] # position at formation time
    tsloc = f['timestep_location'][:] # snapshot where star particle first appears
uIDs = np.unique(hostids)

In [5]:
def get_halo(snapshot, halo_number):
    ts = db.get_timestep(f"{ss_dir}/%{snapshot}")
    # print(f"Retrieved timestep: {ts}")
    return ts.halos.filter_by(halo_number=int(halo_number)).first()

In [6]:
halo_particle_dict = {} # map iords to their unique host IDs
for i, part in enumerate(partids):
    halo_particle_dict[part] = hostids[i]

In [7]:
uIDs

array(['0000_0', '0192_25', '0192_38', '0288_46', '0288_82', '0288_95',
       '0384_39', '0384_68', '0480_15', '0480_19', '0480_43', '0480_45',
       '0480_61', '0576_14', '0576_23', '0672_27', '0672_30', '0672_35',
       '0672_9', '0768_63', '0864_26', '0864_90', '0960_8', '1056_27',
       '1056_364', '1344_29', '1440_25', '1440_49', '1536_25', '1632_6',
       '1728_3', '1728_93', '1824_143', '2112_11', '2304_133', '2304_14',
       '2400_28', '2400_29', '2592_240', '2592_966', '2688_47', '2976_36',
       '3168_138', '3168_27', '3264_53', '3456_270', '3456_56',
       '3552_300', '3648_85', '3648_90', '3744_13', '4032_287', '4096_1',
       '4096_10', '4096_11', '4096_124', '4096_13', '4096_14', '4096_16',
       '4096_192', '4096_2', '4096_20', '4096_208', '4096_218', '4096_22',
       '4096_23', '4096_24', '4096_253', '4096_28', '4096_292', '4096_3',
       '4096_34', '4096_35', '4096_39', '4096_4', '4096_43', '4096_47',
       '4096_49', '4096_5', '4096_50', '4096_6', '4096_6

In [8]:
s = pynbody.load(ss_z0)
h = s.halos(halo_numbers='v1')
main_halo = get_halo('004096', 4)
mask = s.s['amiga.grp'] == main_halo.halo_number

In [9]:
halo_numbers, dbids = main_halo.calculate_for_progenitors("halo_number()", "dbid()")
snapshots = [db.get_halo(dbid).timestep.extension[-6:] for dbid in dbids]
halo_snapshots_dict = {snapshot: halo_number for snapshot, halo_number in zip(snapshots, dbids)}
halo_snapshots_dict.keys()

dict_keys(['004096', '004032', '003936', '003840', '003744', '003648', '003552', '003456', '003360', '003264', '003168', '003072', '002976', '002880', '002784', '002688', '002592', '002496', '002400', '002304', '002208', '002112', '002016', '001920', '001824', '001728', '001632', '001536', '001440', '001344', '001248', '001152', '001056', '000960', '000864', '000768', '000672', '000576', '000480', '000384', '000288', '000192'])

In [10]:
stars_to_consider = s.s['iord'][mask]
unique_starids = np.unique([halo_particle_dict[star] for star in stars_to_consider])
unique_starids

array(['0000_0', '0288_95', '1056_27', '2304_14', '4096_4'], dtype='<U7')

In [10]:
idx = '2304_14'
snapshot, halo_num = idx.split('_')
halo_merger = get_halo(snapshot, int(halo_num))
# halo_numbers = halo.calculate_for_progenitors('halo_number()')

halo_starmask = hostids == idx
all_star_iords = partids[halo_starmask]
all_star_tform = pct[halo_starmask]

# mask for pynbody
# all_starmask = 

In [11]:
ndm, halonums, dbids2 = halo_merger.calculate_for_progenitors('NDM()', 'halo_number()', 'dbid()')
halo_dm_max = db.get_halo(dbids2[np.argmax(ndm)])
halo_dm_max

<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.002016/halo_9' | NDM=882640 Nstar=43532 Ngas=154004>

In [12]:
# get dark matter iords we want
sim = pynbody.load(halo_dm_max.timestep.filename)
mask = sim.dm['amiga.grp'] == int(halo_dm_max.halo_number)
all_dm_iords = sim.dm['iord'][mask]
np.sum(mask)

np.int64(838948)

In [13]:
# --- 3. Initialize data arrays ---
timesteps_to_process = db.get_simulation(ss_dir).timesteps
num_snaps = len(timesteps_to_process)
num_star_particles = len(all_star_iords)
num_dm_particles = len(all_dm_iords)

# Use np.nan to fill arrays. This makes it clear if a particle was not present in a snapshot.
star_iords = np.full((num_snaps, num_star_particles), np.nan)
star_pos = np.full((num_snaps, num_star_particles, 3), np.nan)
star_vel = np.full((num_snaps, num_star_particles, 3), np.nan)
star_mass = np.full((num_snaps, num_star_particles), np.nan)
star_age = np.full((num_snaps, num_star_particles), np.nan)
star_feh = np.full((num_snaps, num_star_particles), np.nan) # Using Fe/H as a single metallicity value

dm_pos = np.full((num_snaps, num_dm_particles, 3), np.nan)
dm_vel = np.full((num_snaps, num_dm_particles, 3), np.nan)
dm_mass = np.full((num_snaps, num_dm_particles), np.nan)

zs = np.zeros(num_snaps)
snaps = [ts.extension for ts in timesteps_to_process]

all_star_iords = np.sort(all_star_iords)
all_dm_iords = np.sort(all_dm_iords)

star_iord_map = {iord: k for k, iord in enumerate(all_star_iords)}
dm_iord_map = {iord: k for k, iord in enumerate(all_dm_iords)}


In [14]:
prev_time = 0
for i, tstep in enumerate(tqdm.tqdm(timesteps_to_process[:2])):
    s = pynbody.load(tstep.filename)
    s.physical_units()
    h = s.halos(halo_numbers='v1')
    halo = db.get_halo(halo_snapshots_dict[tstep.extension[-6:]])
    print(f"Loaded snapshot: {tstep.extension[-6:]}, ", end='')
    zs[i] = s.properties['z']

    # Center the whole simulation on the halo of interest.
    pynbody.analysis.halo.center(h[halo.halo_number], vel=True)
    print(f"Centered on halo: {halo.halo_number}")

    stars_present_mask = pynbody.filt.HighPass('tform', prev_time)
    subs = s.s[stars_present_mask and np.isin(s.s['iord'], all_star_iords)]
    
    iords_in_subs = np.array(subs['iord'])
    k_indices_star = np.array([star_iord_map[iord] for iord in iords_in_subs])

    if len(k_indices_star) > 0:
        star_pos[i, k_indices_star, :] = subs['pos']
        star_vel[i, k_indices_star, :] = subs['vel']
        star_mass[i, k_indices_star] = subs['mass']
        star_age[i, k_indices_star] = subs['age']
        star_feh[i, k_indices_star] = subs['feh']
    print(f"Processed {len(k_indices_star)} star particles in snapshot {tstep.extension[-6:]}")

    subd = s.dm[np.isin(s.dm['iord'], all_dm_iords)]
    iords_in_subd = np.array(subd['iord'])
    k_indices_dm = np.array([dm_iord_map[iord] for iord in iords_in_subd])
    if len(k_indices_dm) > 0:
        dm_pos[i, k_indices_dm, :] = subd['pos']
        dm_vel[i, k_indices_dm, :] = subd['vel']
        dm_mass[i, k_indices_dm] = subd['mass']
    print(f"Processed {len(k_indices_dm)} DM particles in snapshot {tstep.extension[-6:]}")
    prev_time = tstep.time_gyr

  0%|          | 0/2 [00:00<?, ?it/s]

Loaded snapshot: 000192, Centered on halo: 59
Processed 0 star particles in snapshot 000192
Processed 838948 DM particles in snapshot 000192
Loaded snapshot: 000288, Centered on halo: 37
Processed 34 star particles in snapshot 000288
Processed 838948 DM particles in snapshot 000288


In [34]:
stars_present_mask

HighPass('tform', 6.54e-01)

In [30]:
s.s['iord']

SimArray([243770192, 243770193, 243770194, ..., 243788326, 243788327,
          243788328])

In [27]:
all_star_iords

array([243797074, 243797333, 243797620, ..., 245619727, 245619730,
       245619732])

In [28]:
s.s[np.isin(s.s['iord'], all_star_iords)]

<IndexedSubSnap "/home/ns1917/tangos_sims/storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.000192::star:indexed" len=0>

In [ ]:
for i, tstep in enumerate(tqdm.tqdm(timesteps_to_process[:2])):
    s = pynbody.load(tstep.filename)
    s.physical_units()
    h = s.halos(halo_numbers='v1')
    halo = db.get_halo(halo_snapshots_dict[tstep.extension[-6:]])
    print("Loaded snapshot:", tstep.extension[-6:])
    zs[i] = s.properties['z']

    pynbody.analysis.halo.center(h[halo.halo_number])
    print("Centered on halo number:", halo.halo_number)

    # Process Stars
    s_iord_map = {iord: j for j, iord in enumerate(s.s['iord'])}
    current_star_indices_in_snapshot = [s_iord_map.get(iord) for iord in all_star_iords]

    subs = s.s[s.s['iord'].isin(all_star_iords)]
    for k, iord in enumerate(all_star_iords):
        tform = all_star_tform[k]
        snap_idx = current_star_indices_in_snapshot[k]
        if tform <= tstep.time_gyr and snap_idx is not None:
            star_pos[i, k, :] = subs['pos'][snap_idx]
            star_vel[i, k, :] = subs['vel'][snap_idx]
            star_mass[i, :] = subs['mass']
            star_age[i, :] = subs['age']
            star_feh[i, :] = subs['feh']
    print(f"Processed {len(all_star_iords)} star particles in snapshot {tstep.extension[-6:]}")

    # Process DM
    d_iord_map = {iord: j for j, iord in enumerate(s.d['iord'])}
    current_dm_indices_in_snapshot = [d_iord_map.get(iord) for iord in all_dm_iords]

    for k, iord in enumerate(all_dm_iords):
        snap_idx = current_dm_indices_in_snapshot[k]
        if snap_idx is not None:
            dm_pos[i, k, :] = s.d['pos'][snap_idx] 
            dm_vel[i, k, :] = s.d['vel'][snap_idx]
            dm_mass[i, k] = s.d['mass'][snap_idx]
    print(f"Processed {len(all_dm_iords)} DM particles in snapshot {tstep.extension[-6:]}")

  0%|          | 0/2 [00:00<?, ?it/s]

Loaded snapshot: 000192
Centered on halo number: 59
Processed 44422 star particles in snapshot 000192
Processed 838948 DM particles in snapshot 000192
Loaded snapshot: 000288
Centered on halo number: 37
Processed 44422 star particles in snapshot 000288
Processed 838948 DM particles in snapshot 000288


In [16]:
output_filename = os.path.join(outfile_dir, 'uw_boundfrac', f"{ss_dir}_{main_halo.halo_number}_{idx}_particle_data.h5")
output_filename

'/home/ns1917/pynbody/stellarhalo_trace_aw/uw_boundfrac/storm.4096g5HbwK1BH_bn_4_2304_14_particle_data.h5'

In [18]:
output_filename = os.path.join(outfile_dir, 'uw_boundfrac', f"{ss_dir}_{main_halo.halo_number}_{idx}_particle_data.h5")

with h5py.File(output_filename, 'w') as f:
    f.create_dataset('snaps', data=np.bytes_(snaps))
    f.create_dataset('zs', data=zs)
    
    # Star data
    # f.create_dataset('star_iords', data=all_star_iords)
    f.create_dataset('star_pos', data=star_pos)
    f.create_dataset('star_vel', data=star_vel)
    f.create_dataset('star_mass', data=star_mass)
    f.create_dataset('star_age', data=star_age)
    f.create_dataset('star_feh', data=star_feh)
    
    # DM data
    # f.create_dataset('dm_iords', data=all_dm_iords)
    f.create_dataset('dm_pos', data=dm_pos)
    f.create_dataset('dm_vel', data=dm_vel)
    f.create_dataset('dm_mass', data=dm_mass)

print("Done.")

Done.


In [20]:
with h5py.File(output_filename, 'r') as f:
    print(f)

<HDF5 file "storm.4096g5HbwK1BH_bn_4_2304_14_particle_data.h5" (mode r)>


In [20]:
halo.timestep.time_gyr

7.728740336776379

In [24]:
halo = get_halo(192, int(1))
halo

<Halo 'storm.4096g5HbwK1BH_bn/storm.cosmo25cmb.4096g5HbwK1BH.000192/halo_1' | NDM=68256 Nstar=1617 Ngas=44508>

In [ ]:
for idx in unique_starids:
    # get dm iords
    snapshot, halo_num = idx.split('_')
    halo = get_halo(snapshot, int(halo_num))
    halo.calculate_for_progenitors('')